In [3]:
import numpy as np
import cvxpy as cp
import pandas as pd

type = 1  # Change this to 1 for HPWH
if type == 1:
    df = pd.read_csv("output_site_null_60_12.csv")

else:
    df = pd.read_csv("output_site_null_ewh.csv")

# Constants
cp_val = 4181.3
rho = 1000
dt = 8 #what should this be? euler steps per timestep?
T_a = 22 #ambient temperature
V = 250 #(L)
t_in_wet = 62.25 #relative humidity

# Data
P_u = [value if mode == "Upper On" else 0 for value, mode in zip(df["Water Heating Electric Power"], df["Water Heating Mode"])]
P_m = [value if mode == "Heat Pump On" else 0 for value, mode in zip(df["Water Heating Electric Power"], df["Water Heating Mode"])]
P_l = [value if mode == "Lower On" else 0 for value, mode in zip(df["Water Heating Electric Power"], df["Water Heating Mode"])]
T_u = df["T_WH3"].values
T_m = df["T_WH10"].values
T_l = df["T_WH12"].values

W_list = []
z_list = []

for j in range(len(T_u) - 1):  # because of j+1
    Wj = np.zeros((3, 7))  # Each Wj is 3x7

    # Row 1: Upper node (δtpu)
    Wj[0, 2] = dt * (T_u[j] - T_a)       # U_u
    Wj[0, 4] = dt * (T_u[j] - T_m[j])    # K_um
    Wj[0, 6] = rho * cp_val * (T_u[j+1] - T_u[j])  # V_u

    # Row 2: Middle node (δtpm)
    Wj[1, 1] = dt * (T_m[j] - T_a)       # U_m
    Wj[1, 3] = dt * (T_m[j] - T_l[j])    # K_ml
    Wj[1, 4] = dt * (T_m[j] - T_u[j])    # K_um
    Wj[1, 5] = rho * cp_val * (T_m[j+1] - T_m[j])  # V_m

    # Row 3: Lower node
    Wj[2, 0] = dt * (T_l[j] - T_a)       # U_l
    Wj[2, 3] = dt * (T_l[j] - T_m[j])    # K_ml
    Wj[2, 5] = -rho * cp_val * (T_l[j+1] - T_l[j])  # -V_m
    Wj[2, 6] = -rho * cp_val * (T_l[j+1] - T_l[j])  # -V_u

    # curve format: [1, t_in_wet, t_in_wet ** 2, t_lower, t_lower ** 2, t_lower * t_in_wet]
    t_lower = T_l[j]
    cop_coeff = np.array([1.1332, 0.063, -0.0000979, -0.00972, -0.0000214, -0.000686])
    curve_var = np.array([1, t_in_wet, t_in_wet ** 2, t_lower, t_lower ** 2, t_lower * t_in_wet ])
    
    
    # not sure what this should be
    z_j = np.array([
        # δtpu upper node
        dt * (P_u[j]), #Power node 3 * time interval (60s?) #upper element

        # δtpm middle node lower node/ heat pump
        dt * ((P_m[j] * (np.matmul(cop_coeff, curve_var.T))) + P_l[j]),  # dependent on heat pump being on consider starting temperature below 49

        # lower node energy change
        V * rho * cp_val * (T_l[j] - T_l[j+1]) #node 16 cold water input
    ])

    W_list.append(Wj)
    z_list.append(z_j)

# Stack based on time dimension
W = np.vstack(W_list)  # (3N x 7)
z_full = np.hstack(z_list)  # (3N, )

# CVXPY OLS problem
theta = cp.Variable(7) #Ul, Um, Uu, Kml, Kum, Vm, Vu] U - Tank Insulation, K - thermal conductivity, V - node volume
cost = cp.sum_squares(W @ theta - z_full) #z_j = W_j @ theta
problem = cp.Problem(cp.Minimize(cost))
problem.solve()

# Output estimated parameters
param_names = ["Ul", "Um", "Uu", "Kml", "Kum", "Vm", "Vu"]
print("\nThe optimal value is", problem.value)
for name, val in zip(param_names, theta.value):
    print(f"{name} = {val:.4f}")

print("The norm of the residual is ", cp.norm(W @ theta - z_full, p=2).value)



The optimal value is 5.460654078536735e+18
Ul = 2380417.9358
Um = -1703074.2175
Uu = -6305.8085
Kml = 2801612.1501
Kum = 23109.5629
Vm = -1.2902
Vu = 3.4186
The norm of the residual is  2336804244.8045874


In [4]:
#Rearrange energy balance formulas to compute T[j + 1]

def predict_temperatures(T_init, timesteps, theta, draws, dt=1, cp=4181.3, rho=1000, T_a=22, V =250):
    Ul, Um, Uu, Kml, Kum, Vm, Vu = theta
    Vl = V - Vm  - Vu
    Tu_pred, Tm_pred, Tl_pred = [], [], []

    # Starting Temperatures [upper, middle, lower] nodes
    Tu_pred.append(T_init[0])
    Tm_pred.append(T_init[1])
    Tl_pred.append(T_init[2])

    for t in range(timesteps - 1):
        Tu = Tu_pred[-1] #get last predictions
        Tm = Tm_pred[-1]
        Tl = Tl_pred[-1]
        vt = draws[t] #draws

        #rearrange energy balance equations for Euler Update
        # Upper node
        Tu_next = Tu + (1 / (rho * cp * Vu)) * (
            Uu * dt * vt * (T_a - Tu) +
            Kum * dt * vt * (Tm - Tu)
        )

        # Middle node
        Tm_next = Tm + (1 / (rho * cp * Vm)) * (
            Um * dt * vt *  (T_a - Tm) +
            Kml * dt * vt * (Tl - Tm) +
            Kum * dt * vt * (Tu - Tm)
        )

        # Lower node
        Tl_next = Tl - (1 / (rho * cp * (V + Vm + Vu)))  *(
            Ul * dt * vt *  (Tl - T_a) +
            Kml * dt * vt * (Tl - Tm)
        )

        Tu_pred.append(Tu_next)
        Tm_pred.append(Tm_next)
        Tl_pred.append(Tl_next)

    return np.array(Tu_pred) #, np.array(Tm_pred), np.array(Tl_pred)

In [5]:
predictions = predict_temperatures([40,40,40], 8, theta.value, [0, 0, 0, 0, 0, 0, 0, 0])
predictions


array([40., 40., 40., 40., 40., 40., 40., 40.])

## Controller

In [6]:
#8 hour horizon model Data Driven Model
import cvxpy as cp
import numpy as np

i = 8 #horizon
lambda_ = 1
l = 0 #soft constraint
n = 8
T_min = 49
T_max = 60
 

def getOptimalSetpoint(T_init, draws): #input values [T_init (upper, middle, low temperature), draws]

  # Construct the problem.
  #declare variables
  T = cp.Variable(i) #Temperatures 
  s = cp.Variable(i) #setpoint variables
  z = cp.Variable(i) #unmet demand
  objective = cp.Minimize(cp.sum(lambda_ * z**2 + (lambda_*l) * s))

  constraints = [
      s >= 49,
      s <= T_max,
      T == predict_temperatures(T_init, i, theta.value, draws),
      z >= T_min - T,
      z >= 0
      ]
  prob = cp.Problem(objective, constraints)

  # The optimal objective value is returned by `prob.solve()`.
  result = prob.solve()

  # Print results
  print("Optimal T values:", T.value)
  print("Optimal s values:", s.value)
  print("Optimal objective value:", result)

  return s.value

In [7]:
getOptimalSetpoint([40,40,40], [0, 0, 0, 0, 0, 0, 0, 0])

Optimal T values: [40. 40. 40. 40. 40. 40. 40. 40.]
Optimal s values: [52.90340427 52.90340427 52.90340427 52.90340427 52.90340427 52.90340427
 52.90340427 52.90340427]
Optimal objective value: 647.9999707407644


array([52.90340427, 52.90340427, 52.90340427, 52.90340427, 52.90340427,
       52.90340427, 52.90340427, 52.90340427])

In [ ]:
#Implment controller

#run 12 node model

import os
import datetime as dt
import pandas as pd
import numpy as np

from ochre import Dwelling
from ochre.utils import default_input_path  # for using sample files
from ochre import HeatPumpWaterHeater

# Define equipment and simulation parameters
setpoint_default = 51  # in C #alternate b/w 60 and 49
deadband_default = 5.56  # in C
max_setpoint = 60
min_setpoint = 49
water_nodes = 12
run_range = False #runs simulation for a variety of setpoints specified in setpoint_range
simulation_days = 1#220 #172 #220
time_interval = 15 # adjust setpoint every 15 minutes?

site_number = '90023' #90023 #10292#'10441'

flow_data = f'net_flow_{site_number}.csv'

#start_date = dt.datetime(2013, 1, 17, 0, 1) #10441
start_date = dt.datetime(2013, 1, 1, 0, 1) #10292, 90023
#start_date = dt.datetime(2013, 1, 23, 0, 1) #90159
setpoint_range = [setpoint_default]

if run_range == True:
    setpoint_range = np.arange(min_setpoint, max_setpoint, 0.5)

for s in setpoint_range: #run simulation for every setpoint in valid range
    setpoint_default = s
    setpoint = setpoint_default #initialize setpoint
    equipment_args = {
        "start_time": start_date,  # year, month, day, hour, minute
        "time_res": dt.timedelta(minutes=1),
        "duration": dt.timedelta(days=simulation_days),
        "verbosity": 9,  # required to get setpoint and deadband in results
        "save_results": False,  # if True, must specify output_path
        # "output_path": os.getcwd(),        # Equipment parameters
        "Setpoint Temperature (C)": setpoint_default,
        "Tank Volume (L)": 250,
        "Tank Height (m)": 1.22,
        "UA (W/K)": 2.17,
        "HPWH COP (-)": 4.5,
        "water_nodes": water_nodes
    }

    # Create water draw schedule
    times = pd.date_range(
        equipment_args["start_time"],
        equipment_args["start_time"] + equipment_args["duration"],
        freq=equipment_args["time_res"],
        inclusive="left",
    )
    water_draw_magnitude = 12  # L/min
    #withdraw_rate = np.random.choice([0, water_draw_magnitude], p=[0.99, 0.01], size=len(times))
    withdraw_rate = np.loadtxt(f'C:\\Users\\janel\\OneDrive\\Documents\\GitHub\\OCHRE-nrel\\ochre\\defaults\\Input Files\\net_flow_{site_number}.csv')
    withdraw_rate = withdraw_rate[:len(times)]
    current_draws = np.loadtxt(f'C:\\Users\\janel\\OneDrive\\Documents\\GitHub\\OCHRE-nrel\\ochre\\defaults\\Input Files\\net_flow_{site_number}.csv')
    current_draws = current_draws[:len(times)]
    schedule = pd.DataFrame(
        {
            "Water Heating (L/min)": withdraw_rate,
            "Water Heating Setpoint (C)": setpoint_default,  # Setting so that it can reset
            "Water Heating Deadband (C)": deadband_default,  # Setting so that it can reset
            "Zone Temperature (C)": 20,
            "Zone Wet Bulb Temperature (C)": 15,  # Required for HPWH
            "Mains Temperature (C)": 7,
        },
        index=times,
    )

    # Initialize equipment
    hpwh = HeatPumpWaterHeater(schedule=schedule, **equipment_args)

    # Simulate
    data = pd.DataFrame()
    data = {'draw_data' :[], 'setpoint' :[]}
    control_signal = {}
    setpoints = []

    #generate noise for setpoint profile
    for t in hpwh.sim_times:
        # Change setpoint based on hour of day
        #get optimal setpoint

        #Change setpoint every 15 minutes
        current_time = t.minute
        if (current_time % time_interval == 0): #time interval
            current_draws = current_draws[time_interval:]#remove previous draws
            setpoint = getOptimalSetpoint([hpwh.model.next_states[2], hpwh.model.next_states[9]] current_draws) #get node temperatures from 3 and 10
            print(setpoint)
        control_signal = {
            "Setpoint": setpoint
        }

        setpoints.append(setpoint)
        # Run with controls
        _ = hpwh.update(control_signal=control_signal)

    
    df = hpwh.finalize()

    cols_to_plot = [
        "Hot Water Outlet Temperature (C)",
        "Hot Water Average Temperature (C)",
        "Water Heating Deadband Upper Limit (C)",
        "Water Heating Deadband Lower Limit (C)",
        "Water Heating Electric Power (kW)",
        "Hot Water Unmet Demand (kW)",
        "Hot Water Delivered (L/min)",
    ]

    cols_to_save = [
        "Hot Water Outlet Temperature (C)",
        #"T_WH1",
        #"T_WH2"
        "T_WH3",
        #"T_WH7",
        "T_WH10",
        "T_WH12"
        #"T_AMB",
        "Water Heating Electric Power (kW)",
    ]


      # For the DataFrame, select columns and calculate the rolling average for each column
    to_save = df[cols_to_save].rolling(window=15).mean()
    # Calculate the rolling average for 'setpoints' with window size 15
    avg_setpoints = np.convolve(setpoints, np.ones(15)/15, 'same')
    avg_setpoints = avg_setpoints[14::15]
    to_save = to_save[14::15]
    to_save["Setpoint"] = pd.Series(avg_setpoints, index=to_save.index)



    to_save = to_save[:-1] 
    to_save.to_csv(f'output_site_{site_number}_physics.csv', mode='a', header=True, index=False)


# Validation

In [3]:
import pandas as pd
from sklearn.metrics import mean_squared_error

# Choose data based on type

if type == 1:  # HPWH
    val_data = pd.read_csv("output_site_90023.csv")
else:  # EWH
    val_data = pd.read_csv("output_site_90023_ewh.csv")

# Parameters
window_size = 8
predictions = []
true_values = []

val_data = val_data[val_data['Hot Water Outlet Temperature (C)'] >= 40] #filter for above 40 temperatures

# Sanity check, iterate through idle
for index in range(len(val_data) - window_size):
    t_u = val_data['T_WH3'].iloc[index]
    t_m = val_data['T_WH10'].iloc[index]
    t_l = val_data['T_WH12'].iloc[index]
    draws = val_data['Draw Data'].iloc[index: index + window_size].reset_index(drop=True)
    
    y_pred = predict_temperatures([t_u, t_m, t_l], window_size, theta.value, draws)
    y_true = [
        val_data['T_WH3'].iloc[index  : index  + window_size].values,
        val_data['T_WH10'].iloc[index : index  + window_size].values,
        val_data['T_WH12'].iloc[index : index + window_size].values
    ]
    
    predictions.append(y_pred)
    true_values.append(y_true)
    index += 1

# Flatten lists for metrics, depending on model output format
# Example: compute MSE for each level
import numpy as np

y_pred_array = np.array(predictions)  # shape: (n_sam09ples, 3, window_size) or similar
y_true_array = np.array(true_values)

mse_upper = mean_squared_error(y_true_array[:, 0, :].flatten(), y_pred_array[:, 0, :].flatten())
mse_middle = mean_squared_error(y_true_array[:, 1, :].flatten(), y_pred_array[:, 1, :].flatten())
mse_lower = mean_squared_error(y_true_array[:, 2, :].flatten(), y_pred_array[:, 2, :].flatten())

print(f"MSE Upper: {mse_upper:.3f}")
print(f"MSE Middle: {mse_middle:.3f}")
print(f"MSE Lower: {mse_lower:.3f}")

print(f"RMSE Upper: {np.sqrt(mse_upper)}")
print(f"RMSE Middle: {np.sqrt(mse_middle)}")
print(f"RMSE Lower: {np.sqrt(mse_lower)}")


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [ ]:
predictions

array([60.52664956, 60.52664956, 60.52664956, 60.52664956, 60.52664956,
       60.52664956, 60.52664956, 60.52664956])

In [28]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display
import matplotlib.dates as mdates

#Predictions for 1 timestep

# Constants
intervals = 96  # 1440 minutes / 15 min interval = 96 per day
year = "2013"
window = 8


# Create datetime labels for x-axis
start_time = datetime(2013, 1, 1, 0, 0)
end_time = datetime(2013, 1, 1, 23, 59)
datetime_list = pd.date_range(start=start_time, end=end_time, freq="15min")
time_labels = [dt.strftime("%H:%M") for dt in datetime_list]

import copy
# Interactive plot function
def plot_predictions_by_hour(day, hour):
    plt.figure(figsize=(10, 5))
    
    start_idx = intervals * day
    
    x_day = (val_data[start_idx : start_idx + intervals])
    predictions_day = predictions[start_idx : start_idx + intervals]
    actual_day = true_values[start_idx : start_idx + intervals]
    draw_data_day = x_day['Draw Data']
    power_day = x_day['Average Electric Power']


    time_idx = hour * 4  # 4 intervals per hour

    pred_times = time_labels[time_idx : time_idx + 8]
    y_pred = predictions_day[time_idx] #returns predictions for top, middle, and bottom nodes
    y_true = actual_day[time_idx] 


    draw_slice = draw_data_day[time_idx : time_idx + 8]
    power_slice = power_day[time_idx : time_idx + 8] * 100

    # Convert day number to readable date
    day_num = str((day + 17) % 365).rjust(3, '0')
    date_label = datetime.strptime(year + "-" + day_num, "%Y-%j").strftime("%m-%d-%Y")

    # Primary axis for temperatures
    fig, ax1 = plt.subplots(figsize=(10, 5))

    ax1.plot(pred_times, y_true[0], label="Actual Outlet Temp", color="#D95F02", marker='o')
    ax1.plot(pred_times, y_pred, label="Predicted Outlet Temp", color="#FC8D62", marker='x', linestyle="--")
    ax1.set_ylabel("Temperature (°C)", color="#D95F02", fontsize=14)
    ax1.set_title("Physics-Based Model Predictions", fontsize=14)
    #ax1.set_title(f"{date_label} @ {time_labels[time_idx]} — Predictions for t+0 to t+7")
    ax1.tick_params(axis='y', colors="#D95F02")
    ax1.set_xticks(pred_times)
    ax1.set_xticklabels(pred_times, rotation=45)
    ax1.set_xlabel("Time of Day", fontsize=14)
    ax1.grid(True)

    # Secondary axis for water draw
        # Secondary axis for water draw
    ax2 = ax1.twinx()
    ax2.plot(pred_times, draw_slice, label="Water Drawn (L)", color="blue", linestyle="dotted", marker='s')
    #ax2.plot(pred_times, power_slice, label="Average Power (kWH)", color = "green")
    ax2.set_ylabel("Water Drawn (L)", color="blue", fontsize=14)
    ax2.tick_params(axis='y', colors="blue")
    ax2.set_ylim(0, 200)  # Ensure 0 is at the bottom


    # Combined legend
    lines_1, labels_1 = ax1.get_legend_handles_labels()
    lines_2, labels_2 = ax2.get_legend_handles_labels()
    #ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper left', bbox_to_anchor=(1.05, 1), fontsize=14)

    plt.tight_layout()
    plt.show()
    print(power_slice/100)

# Slider widgets
day_slider = widgets.IntSlider(value=25, min=0, max=len(val_data)//intervals - 1, step=1, description="Day:")
hour_slider = widgets.IntSlider(value=0, min=0, max=23, step=1, description="Hour:")

# Display interactive plot
widgets.interact(plot_predictions_by_hour, day=day_slider, hour=hour_slider)



interactive(children=(IntSlider(value=25, description='Day:', max=9676), IntSlider(value=0, description='Hour:…

<function __main__.plot_predictions_by_hour(day, hour)>

Experiments
1) Perfect foresight of future hourly average water use patterns (accurate in daily shape but not granular)
    36, 54, and 72 gal/day draw profilesout
    Simulation based test 28.8 to 72 gal/day

2) CasADi python optimization package, IPOPT solver

3) Use cvxpy for OLS, etc.